In [4]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import 

from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [6]:
dataset_dir = './aclImdb'
os.listdir(dataset_dir)

['test', 'train']

In [7]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [8]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [9]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclimdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [12]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[ 0.01746312,  0.01151482, -0.02747941, -0.0277862 ,  0.04785163],
       [ 0.01872969,  0.01693792,  0.03065981, -0.01543211,  0.03538997],
       [ 0.03064852,  0.04989347,  0.00584779,  0.03331008, -0.04442445]],
      dtype=float32)

In [13]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

TensorShape([2, 3, 5])

In [14]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)

text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [17]:
embedding_dim=16

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim, name="Embedding"),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1)
])

In [18]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [19]:
model.compile(optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [20]:
model.fit(
    train_ds,
    validation_data = val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
 1/20 [>.............................] - ETA: 0s - loss: 0.6933 - accuracy: 0.5127WARNING:tensorflow:From C:\Users\jedgo\Python\Python38\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
20/20 [==============================] - 20s 1s/step - loss: 0.6923 - accuracy: 0.5028 - val_loss: 0.6906 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 107ms/step - loss: 0.6876 - accuracy: 0.5028 - val_loss: 0.6839 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 107ms/step - loss: 0.6784 - accuracy: 0.5028 - val_loss: 0.6724 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 107ms/step - loss: 0.6636 - accuracy: 0.5028 - val_loss: 0.6553 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 107ms/ste

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 100)               0         
_________________________________________________________________
Embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [22]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [24]:
vocab = vectorize_layer.get_vocabulary()
print(vocab[:10])

weights = model.get_layer('Embedding').get_weights()[0]
print(weights.shape)

['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it']
(10000, 16)


In [25]:
  out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
  out_m = io.open('meta.tsv', 'w', encoding='utf-8')

  for num, word in enumerate(vocab):
    if num == 0: continue # skip padding token from vocab
    vec = weights[num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_v.close()
  out_m.close()

  try:
    from google.colab import files
  except ImportError:
    pass
  else:
    files.download('vecs.tsv')
    files.download('meta.tsv')